### Terminology术语



$o_t$ observation

$a_t$ action

$\pi_{\theta}(a_t|o_t)$ policy 

$\pi_{\theta}(a_t|s_t)$ policy (fully observed) 

state和observation有什么区别？

- observation就是你观察到的，它可能包含推理出状态的完整信息，也可能不;

- state表示所有的信息，也就是fully observed.状态符合马尔可夫性质


### imitation learning

训练数据之间没有独立同分布假设，这与监督学习很大程度上不同，之前的输入会影响后面的状态

- 以更聪明的方式收集数据
- 使用更加强大的模型
- 使用多任务学习

### Why behavior cloning fail？

使用监督学习常用的衡量标准不适用